In [38]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [39]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [40]:
import random

In [41]:
# Create list size of 500 with integer between 0 and 9.
list = [random.randrange(10) for x in range(500)]

In [42]:
listrdd = sc.parallelize(list, 5)

In [68]:
pairs = listrdd.map(lambda x : (x,x*x))

In [69]:
pairs.collect()

[(5, 25),
 (4, 16),
 (5, 25),
 (2, 4),
 (1, 1),
 (5, 25),
 (1, 1),
 (8, 64),
 (9, 81),
 (1, 1),
 (7, 49),
 (3, 9),
 (1, 1),
 (4, 16),
 (0, 0),
 (0, 0),
 (3, 9),
 (6, 36),
 (4, 16),
 (3, 9),
 (0, 0),
 (0, 0),
 (2, 4),
 (7, 49),
 (8, 64),
 (1, 1),
 (7, 49),
 (4, 16),
 (1, 1),
 (5, 25),
 (5, 25),
 (7, 49),
 (5, 25),
 (1, 1),
 (6, 36),
 (3, 9),
 (1, 1),
 (5, 25),
 (0, 0),
 (7, 49),
 (9, 81),
 (5, 25),
 (6, 36),
 (0, 0),
 (1, 1),
 (8, 64),
 (1, 1),
 (5, 25),
 (4, 16),
 (4, 16),
 (6, 36),
 (0, 0),
 (8, 64),
 (5, 25),
 (6, 36),
 (0, 0),
 (3, 9),
 (7, 49),
 (1, 1),
 (9, 81),
 (9, 81),
 (7, 49),
 (6, 36),
 (8, 64),
 (3, 9),
 (9, 81),
 (3, 9),
 (4, 16),
 (6, 36),
 (8, 64),
 (9, 81),
 (7, 49),
 (7, 49),
 (8, 64),
 (3, 9),
 (2, 4),
 (3, 9),
 (0, 0),
 (8, 64),
 (1, 1),
 (9, 81),
 (6, 36),
 (8, 64),
 (0, 0),
 (8, 64),
 (0, 0),
 (7, 49),
 (0, 0),
 (0, 0),
 (3, 9),
 (7, 49),
 (2, 4),
 (9, 81),
 (8, 64),
 (8, 64),
 (7, 49),
 (5, 25),
 (3, 9),
 (4, 16),
 (9, 81),
 (1, 1),
 (3, 9),
 (2, 4),
 (8, 64),
 (8

In [70]:
reduced = pairs.reduceByKey(lambda x1,x2  : x1+x2)

In [71]:
#mapPartitions() : Return a new RDD by applying a function to each partition of this RDD.
finalRDD = reduced.mapPartitions(lambda itr : ["Key = " + str(key) +", Value = " + str(val) for (key,val) in itr])

In [72]:
finalRDD.collect()

['Key = 0, Value = 0',
 'Key = 5, Value = 850',
 'Key = 1, Value = 60',
 'Key = 6, Value = 1836',
 'Key = 2, Value = 168',
 'Key = 7, Value = 2744',
 'Key = 8, Value = 3968',
 'Key = 3, Value = 486',
 'Key = 9, Value = 4050',
 'Key = 4, Value = 720']

In [73]:
print finalRDD.toDebugString()

(5) PythonRDD[60] at collect at <ipython-input-72-3fb147ed11e7>:1 []
 |  MapPartitionsRDD[59] at mapPartitions at PythonRDD.scala:427 []
 |  ShuffledRDD[58] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[57] at reduceByKey at <ipython-input-70-c5e66787794d>:1 []
    |  PythonRDD[56] at reduceByKey at <ipython-input-70-c5e66787794d>:1 []
    |  ParallelCollectionRDD[22] at parallelize at PythonRDD.scala:480 []


In [74]:
ordered_finalRDD = pairs.reduceByKey(lambda x1,x2  : x1+x2).sortByKey()

In [75]:
ordered_finalRDD.collect()

[(0, 0),
 (1, 60),
 (2, 168),
 (3, 486),
 (4, 720),
 (5, 850),
 (6, 1836),
 (7, 2744),
 (8, 3968),
 (9, 4050)]

In [76]:
print ordered_finalRDD.toDebugString()

(5) PythonRDD[71] at collect at <ipython-input-75-6454b7f1c2c7>:1 []
 |  MapPartitionsRDD[70] at mapPartitions at PythonRDD.scala:427 []
 |  ShuffledRDD[69] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[68] at sortByKey at <ipython-input-74-6f0a229ec686>:1 []
    |  PythonRDD[67] at sortByKey at <ipython-input-74-6f0a229ec686>:1 []
    |  MapPartitionsRDD[64] at mapPartitions at PythonRDD.scala:427 []
    |  ShuffledRDD[63] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(5) PairwiseRDD[62] at reduceByKey at <ipython-input-74-6f0a229ec686>:1 []
       |  PythonRDD[61] at reduceByKey at <ipython-input-74-6f0a229ec686>:1 []
       |  ParallelCollectionRDD[22] at parallelize at PythonRDD.scala:480 []


In [77]:
coalesce = ordered_finalRDD.coalesce(3)

In [78]:
print coalesce.toDebugString()

(3) CoalescedRDD[72] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  PythonRDD[71] at collect at <ipython-input-75-6454b7f1c2c7>:1 []
 |  MapPartitionsRDD[70] at mapPartitions at PythonRDD.scala:427 []
 |  ShuffledRDD[69] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[68] at sortByKey at <ipython-input-74-6f0a229ec686>:1 []
    |  PythonRDD[67] at sortByKey at <ipython-input-74-6f0a229ec686>:1 []
    |  MapPartitionsRDD[64] at mapPartitions at PythonRDD.scala:427 []
    |  ShuffledRDD[63] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(5) PairwiseRDD[62] at reduceByKey at <ipython-input-74-6f0a229ec686>:1 []
       |  PythonRDD[61] at reduceByKey at <ipython-input-74-6f0a229ec686>:1 []
       |  ParallelCollectionRDD[22] at parallelize at PythonRDD.scala:480 []


In [79]:
repartition = ordered_finalRDD.repartition(3)

In [80]:
print repartition.toDebugString()

(3) MapPartitionsRDD[77] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  CoalescedRDD[76] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  ShuffledRDD[75] at coalesce at NativeMethodAccessorImpl.java:0 []
 +-(5) MapPartitionsRDD[74] at coalesce at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[73] at RDD at PythonRDD.scala:48 []
    |  MapPartitionsRDD[70] at mapPartitions at PythonRDD.scala:427 []
    |  ShuffledRDD[69] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(5) PairwiseRDD[68] at sortByKey at <ipython-input-74-6f0a229ec686>:1 []
       |  PythonRDD[67] at sortByKey at <ipython-input-74-6f0a229ec686>:1 []
       |  MapPartitionsRDD[64] at mapPartitions at PythonRDD.scala:427 []
       |  ShuffledRDD[63] at partitionBy at NativeMethodAccessorImpl.java:0 []
       +-(5) PairwiseRDD[62] at reduceByKey at <ipython-input-74-6f0a229ec686>:1 []
          |  PythonRDD[61] at reduceByKey at <ipython-input-74-6f0a229ec686>:1 []
          |  ParallelCollection

In [82]:
print repartition.toDebugString()

sc.setCheckpointDir("checkpoint")
repartition.checkpoint() # all references to its parent RDDs will be removed.
print repartition.isCheckpointed() # not checkpointed until action()

repartition.count()
print repartition.isCheckpointed()
print repartition.getCheckpointFile()

(3) MapPartitionsRDD[77] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  CoalescedRDD[76] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  ShuffledRDD[75] at coalesce at NativeMethodAccessorImpl.java:0 []
 +-(5) MapPartitionsRDD[74] at coalesce at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[73] at RDD at PythonRDD.scala:48 []
    |  MapPartitionsRDD[70] at mapPartitions at PythonRDD.scala:427 []
    |  ShuffledRDD[69] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(5) PairwiseRDD[68] at sortByKey at <ipython-input-74-6f0a229ec686>:1 []
       |  PythonRDD[67] at sortByKey at <ipython-input-74-6f0a229ec686>:1 []
       |  MapPartitionsRDD[64] at mapPartitions at PythonRDD.scala:427 []
       |  ShuffledRDD[63] at partitionBy at NativeMethodAccessorImpl.java:0 []
       +-(5) PairwiseRDD[62] at reduceByKey at <ipython-input-74-6f0a229ec686>:1 []
          |  PythonRDD[61] at reduceByKey at <ipython-input-74-6f0a229ec686>:1 []
          |  ParallelCollection

In [84]:
print repartition.toDebugString()

(3) MapPartitionsRDD[77] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  ReliableCheckpointRDD[80] at count at <ipython-input-82-caee0cbe841a>:8 []
